N. Tarr 7/19/2018

Python 2.7

Description: Match SGCN species to GAP species.  Begin by matching on TSN codes.  Running this code requires access to the gapproduction package, which requires arcpy and connection to the NCSU GAP databases.  

Note: It is necessary to have access to a info from NCSU GAP databases on gap species name-ITIS crosswalk info.  Here, it is accessed with a csv file.  Each GAP species is linked with an ITIS TSN.  However, some GAP species are not recognized by ITIS or direct matches will require extensive review of literature.  GAP species may be linked to a "parent" TSN (e.g. for the Genus) in such cases.

In [22]:
import pandas as pd
import sys
import pprint
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/bin64')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/ArcPy')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/ArcToolBox/Scripts')
sys.path.append('P:/Proj3/USGap/Scripts/GAPAnalysis')
sys.path.append('P:/Proj3/USGap/Scripts/GAPProduction')
sys.path.append('P:/Proj3/USGap/Scripts/')
import gapproduction as gp
import pymongo

Functions and variables for connecting to bis database

In [ ]:
host="54.91.95.139"
port=27017
username=raw_input("username: ")
password=raw_input("password: ")

def _connect_mongodb(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = pymongo.MongoClient(mongo_uri)
    else:
        conn = pymongo.MongoClient(host, port)
        
    return conn[db]

def get_collection_cursor(db, collection, host, port, username, password, query={}):
    """ Read from Mongo """

    # Connect to MongoDB
    db = _connect_mongodb(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)
    
    return cursor

## Create a dictionary to hold SGCN - GAP matches

In [44]:
matches = {}
non_matches = {}

## Read in GAP Scientific Names, TSN codes, and species codes

In [ ]:
# Read in csv of gap species, species codes, and TSN codes and notes
gapSpecies = pd.read_csv("T:/SALCC/GAPspecies2.csv")
gapSpecies.drop(["scientific_name", "common_name", "strComName"],inplace=True, axis=1)
gapSpecies.drop(columns=["Unnamed: 0"],inplace=True, axis=1)
gapSpecies['intITIScode'].fillna(0, inplace=True)
gapSpecies['intITIScode'] = [int(i) for i in gapSpecies['intITIScode']]

# Print first 5 records
gapSpecies.head()

## Read in SGCN Scientific Names and TSN codes, attempt to match with GAP
Define the search query for filtering out unwanted records from the bis database

In [39]:
query = {"Taxonomic Group": {"$in": ["Birds", "Reptiles", "Amphibians", "Mammals"]}}

Process each document in the collection. for each document/record, first check to see if the SGCN TSN is associated with a GAP species.  If it is, collect that record and match.  Note that there are some one-to-many matches on tsn (one SGCN TSN to many GAP species; sometimes due to subspecies).  Those aren't resolved here; they count as non-matches.

In [40]:
# Get a cursor for the SGCN Synthesis collection from bis
SGCNcursor = get_collection_cursor(db='bis', collection='SGCN Synthesis', query=query, host=host, port=port, 
                                   username=username, password=password)

# Loop through SGCN's and collect matches on TSN
for x in SGCNcursor:
    tsn = int(x['Taxonomic Authority ID'][-6:])
    sciname = x['Taxonomy'][-1]['name']
    if int(tsn) in set(gapSpecies.intITIScode):
        GAPtsnDF = gapSpecies[gapSpecies['intITIScode'] == tsn]
        if len(GAPtsnDF) == 1:
            spInfo = {}
            spInfo["SGCN_id"] = sciname
            spInfo["SGCN-ITIS_TSN"] = tsn
            spInfo["SGCN_commmon_name"] = x['Common Name']
            spInfo["GAP-ITIS_TSN"] = tsn
            spInfo["GAP_scientific_name"] = GAPtsnDF["strSciName"].unique()[0]
            spInfo["GAP_common_name"] = gp.gapdb.NameCommon(GAPtsnDF["strUC"].unique()[0])
            spInfo["GAP_species_code"] = GAPtsnDF["strUC"].unique()[0]
            spInfo["GAP-ITIS_match_notes"] = GAPtsnDF["strGapITISmatch"].unique()[0]
            spInfo["GAP_taxa_notes"] = GAPtsnDF["memTaxaNotes"].unique()[0]
            spInfo["notes"] = ""
            matches[sciname] = spInfo
        else:
            non_matches[sciname] = spInfo
    else:
        spInfo = {}
        spInfo["SGCN_scientific_name"] = sciname
        spInfo["SGCN-ITISTSN"] = tsn
        spInfo["SGCN_commmon_name"] = x['Common Name']
        non_matches[sciname] = spInfo


# Display results
Show the number of records in each dictionary

In [41]:
print("SGCN Synthesis documents: " + str(len(list(get_collection_cursor(db='bis', collection='SGCN Synthesis', 
                                                                        query=query, host=host, port=port,
                                                                        username=username, password=password)))))
print("GAP species: 1719")
print("Matches: " + str(len(matches.keys())))
print("Non-matches: " + str(len(non_matches.keys())))

SGCN Synthesis documents: 2449
GAP species: 1719
Matches: 1329
Non-matches: 1120


# Non-matches

In [42]:
pprint.pprint(non_matches)

{u'Acanthis flammea': {'SGCN-ITISTSN': 179241,
                       'SGCN_commmon_name': u'Common Redpoll',
                       'SGCN_scientific_name': u'Acanthis flammea'},
 u'Acanthis hornemanni': {'SGCN-ITISTSN': 179238,
                          'SGCN_commmon_name': u'Arctic Redpoll',
                          'SGCN_scientific_name': u'Acanthis hornemanni'},
 u'Accipiter gentilis apache': {'SGCN-ITISTSN': 824282,
                                'SGCN_commmon_name': u'Apache Northern Goshawk',
                                'SGCN_scientific_name': u'Accipiter gentilis apache'},
 u'Accipiter gentilis atricapillus': {'SGCN-ITISTSN': 175301,
                                      'SGCN_commmon_name': u'Northern Goshawk',
                                      'SGCN_scientific_name': u'Accipiter gentilis atricapillus'},
 u'Accipiter gentilis laingi': {'SGCN-ITISTSN': 175302,
                                'SGCN_commmon_name': u'Queen Charlotte Goshawk',
                            

 u'Anolis cooki': {'SGCN-ITISTSN': 173886,
                   'SGCN_commmon_name': u"Cook's Anole",
                   'SGCN_scientific_name': u'Anolis cooki'},
 u'Anolis cristatellus': {'SGCN-ITISTSN': 173887,
                          'SGCN_commmon_name': u'Crested Anole',
                          'SGCN_scientific_name': u'Anolis cristatellus'},
 u'Anolis cuvieri': {'SGCN-ITISTSN': 173888,
                     'SGCN_commmon_name': u'Puerto Rican Giant Anole',
                     'SGCN_scientific_name': u'Anolis cuvieri'},
 u'Anolis occultus': {'SGCN-ITISTSN': 173898,
                      'SGCN_commmon_name': u'Pygmy Anole',
                      'SGCN_scientific_name': u'Anolis occultus'},
 u'Anolis poncensis': {'SGCN-ITISTSN': 173899,
                       'SGCN_commmon_name': u'Dryland Grass Anole',
                       'SGCN_scientific_name': u'Anolis poncensis'},
 u'Anolis pulchellus': {'SGCN-ITISTSN': 173901,
                        'SGCN_commmon_name': u'Common Grass Anol

 u'Branta canadensis occidentalis': {'SGCN-ITISTSN': 175006,
                                     'SGCN_commmon_name': u'Dusky Canada Goose',
                                     'SGCN_scientific_name': u'Branta canadensis occidentalis'},
 u'Branta hutchinsii leucopareia': {'SGCN-ITISTSN': 714726,
                                    'SGCN_commmon_name': u'Aleutian Canada goose (Semidi Island population only)',
                                    'SGCN_scientific_name': u'Branta hutchinsii leucopareia'},
 u'Branta hutchinsii minima': {'SGCN-ITISTSN': 714727,
                               'SGCN_commmon_name': u'Cackling Cackling Goose',
                               'SGCN_scientific_name': u'Branta hutchinsii minima'},
 u'Branta hutchinsii taverneri': {'SGCN-ITISTSN': 714728,
                                  'SGCN_commmon_name': u'Taverner\xe2\x80\x99s Cackling Goose',
                                  'SGCN_scientific_name': u'Branta hutchinsii taverneri'},
 u'Branta sandvicensis': {

                                'SGCN_scientific_name': u'Cervus elaphus canadensis'},
 u'Chaetodipus californicus femoralis': {'SGCN-ITISTSN': 900820,
                                         'SGCN_commmon_name': u'Dulzura Pocket Mouse',
                                         'SGCN_scientific_name': u'Chaetodipus californicus femoralis'},
 u'Chaetodipus fallax fallax': {'SGCN-ITISTSN': 900826,
                                'SGCN_commmon_name': u'Northwestern San Diego Pocket Mouse',
                                'SGCN_scientific_name': u'Chaetodipus fallax fallax'},
 u'Chaetodipus fallax pallidus': {'SGCN-ITISTSN': 900830,
                                  'SGCN_commmon_name': u'Pallid San Diego Pocket Mouse',
                                  'SGCN_scientific_name': u'Chaetodipus fallax pallidus'},
 u'Charadrius alexandrinus': {'SGCN-ITISTSN': 176510,
                              'SGCN_commmon_name': u'Snowy Plover',
                              'SGCN_scientific_name': u'Char

                                      'SGCN_scientific_name': u'Conepatus leuconotus leuconotus'},
 u'Conuropsis carolinensis': {'SGCN-ITISTSN': 177407,
                              'SGCN_commmon_name': u'Carolina Parakeet',
                              'SGCN_scientific_name': u'Conuropsis carolinensis'},
 u'Corvus corax kamtschaticus': {'SGCN-ITISTSN': 924235,
                                 'SGCN_commmon_name': u'Common Raven',
                                 'SGCN_scientific_name': u'Corvus corax kamtschaticus'},
 u'Corvus hawaiiensis': {'SGCN-ITISTSN': 179742,
                         'SGCN_commmon_name': u"'Alala",
                         'SGCN_scientific_name': u'Corvus hawaiiensis'},
 u'Corvus kubaryi': {'SGCN-ITISTSN': 179744,
                     'SGCN_commmon_name': u'Mariana Crow',
                     'SGCN_scientific_name': u'Corvus kubaryi'},
 u'Corvus leucognaphalus': {'SGCN-ITISTSN': 179745,
                            'SGCN_commmon_name': u'White-necked Crow',
   

...

                         'SGCN_scientific_name': u'Setophaga petechia'},
 u'Setophaga petechia gundlachi': {'SGCN-ITISTSN': 950170,
                                   'SGCN_commmon_name': u'Cuban Yellow Warbler',
                                   'SGCN_scientific_name': u'Setophaga petechia gundlachi'},
 u'Setophaga pinus': {'SGCN-ITISTSN': 950045,
                      'SGCN_commmon_name': u'Pine Warbler',
                      'SGCN_scientific_name': u'Setophaga pinus'},
 u'Setophaga pitiayumi': {'SGCN-ITISTSN': 950034,
                          'SGCN_commmon_name': u'Tropical Parula',
                          'SGCN_scientific_name': u'Setophaga pitiayumi'},
 u'Setophaga striata': {'SGCN-ITISTSN': 950041,
                        'SGCN_commmon_name': u'Blackpoll Warbler',
                        'SGCN_scientific_name': u'Setophaga striata'},
 u'Setophaga tigrina': {'SGCN-ITISTSN': 950031,
                        'SGCN_commmon_name': u'Cape May Warbler',
                        'SGCN

 u'Spinus psaltria': {'SGCN-ITISTSN': 179254,
                      'SGCN_commmon_name': u'Lesser Goldfinch',
                      'SGCN_scientific_name': u'Spinus psaltria'},
 u'Spinus tristis': {'SGCN-ITISTSN': 179249,
                     'SGCN_commmon_name': u'American Goldfinch',
                     'SGCN_scientific_name': u'Spinus tristis'},
 u'Spizella breweri breweri': {'SGCN-ITISTSN': 179442,
                               'SGCN_commmon_name': u"Brewer's Sparrow",
                               'SGCN_scientific_name': u'Spizella breweri breweri'},
 u'Spizelloides arborea': {'SGCN-ITISTSN': 997961,
                           'SGCN_commmon_name': u'American Tree Sparrow',
                           'SGCN_scientific_name': u'Spizelloides arborea'},
 u'Stelgidopteryx ruficollis': {'SGCN-ITISTSN': 178440,
                                'SGCN_commmon_name': u'Southern Rough-winged Swallow',
                                'SGCN_scientific_name': u'Stelgidopteryx ruficollis'},
 u'

 u'Terrapene carolina': {'SGCN-ITISTSN': 180461,
                         'SGCN_commmon_name': u'Pygmy Killer Whale',
                         'SGCN_scientific_name': u'Feresa attenuata'},
 u'Terrapene carolina carolina': {'SGCN-ITISTSN': 173777,
                                  'SGCN_commmon_name': u'Eastern Box Turtle',
                                  'SGCN_scientific_name': u'Terrapene carolina carolina'},
 u'Terrapene carolina major': {'SGCN-ITISTSN': 208598,
                               'SGCN_commmon_name': u'Gulf Coast Box Turtle',
                               'SGCN_scientific_name': u'Terrapene carolina major'},
 u'Terrapene carolina triunguis': {'SGCN-ITISTSN': 208600,
                                   'SGCN_commmon_name': u'Three-toed Box Turtle',
                                   'SGCN_scientific_name': u'Terrapene carolina triunguis'},
 u'Terrapene ornata luteola': {'SGCN-ITISTSN': 208603,
                               'SGCN_commmon_name': u'Desert Box Turtle',
   

 u'Troglodytes pacificus kiskensis': {'SGCN-ITISTSN': 925953,
                                      'SGCN_commmon_name': u'Pacific Wren',
                                      'SGCN_scientific_name': u'Troglodytes pacificus kiskensis'},
 u'Troglodytes pacificus meligerus': {'SGCN-ITISTSN': 925952,
                                      'SGCN_commmon_name': u'Attu Winter Wren',
                                      'SGCN_scientific_name': u'Troglodytes pacificus meligerus'},
 u'Troglodytes pacificus ochroleucus': {'SGCN-ITISTSN': 925958,
                                        'SGCN_commmon_name': u'Pacific Wren',
                                        'SGCN_scientific_name': u'Troglodytes pacificus ochroleucus'},
 u'Troglodytes pacificus petrophilus': {'SGCN-ITISTSN': 925956,
                                        'SGCN_commmon_name': u'Unalaska Winter Wren',
                                        'SGCN_scientific_name': u'Troglodytes pacificus petrophilus'},
 u'Troglodytes pacificus

                                'SGCN_scientific_name': u'Vulpes vulpes cascadensis'},
 u'Xantusia vigilis': {'SGCN-ITISTSN': 178978,
                       'SGCN_commmon_name': u'Redstarts',
                       'SGCN_scientific_name': u'Setophaga'},
 u'Xantusia vigilis vigilis': {'SGCN-ITISTSN': 208994,
                               'SGCN_commmon_name': u'Desert Night Lizard',
                               'SGCN_scientific_name': u'Xantusia vigilis vigilis'},
 u'Xema sabini': {'SGCN-ITISTSN': 176866,
                  'SGCN_commmon_name': u"Sabine's Gull",
                  'SGCN_scientific_name': u'Xema sabini'},
 u'Xerospermophilus tereticaudus chlorus': {'SGCN-ITISTSN': 931243,
                                            'SGCN_commmon_name': u'Coachella Valley round-tailed ground squirrel',
                                            'SGCN_scientific_name': u'Xerospermophilus tereticaudus chlorus'},
 u'Zenaida aurita': {'SGCN-ITISTSN': 177118,
                     'SGCN_commmo

# Matches

In [30]:
pprint.pprint(matches)

{u'Accipiter cooperii': {'GAP-ITIS_TSN': 175309,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u"Cooper's Hawk",
                         'GAP_scientific_name': 'Accipiter cooperii',
                         'GAP_species_code': 'bCOHAx',
                         'GAP_taxa_notes': nan,
                         'SGCN-ITIS_TSN': 175309,
                         'SGCN_commmon_name': u"Cooper's Hawk",
                         'SGCN_id': u'Accipiter cooperii',
                         'notes': ''},
 u'Accipiter gentilis': {'GAP-ITIS_TSN': 175300,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u'Northern Goshawk',
                         'GAP_scientific_name': 'Accipiter gentilis',
                         'GAP_species_code': 'bNOGOx',
                         'GAP_taxa_notes': nan,
                         'SGCN-ITIS_TSN': 175300,
                         'S

                           'SGCN-ITIS_TSN': 179340,
                           'SGCN_commmon_name': u"Henslow's Sparrow",
                           'SGCN_id': u'Ammodramus henslowii',
                           'notes': ''},
 u'Ammodramus leconteii': {'GAP-ITIS_TSN': 179345,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u"Le Conte's Sparrow",
                           'GAP_scientific_name': 'Ammodramus leconteii',
                           'GAP_species_code': 'bLCSPx',
                           'GAP_taxa_notes': nan,
                           'SGCN-ITIS_TSN': 179345,
                           'SGCN_commmon_name': u"Le Conte's Sparrow",
                           'SGCN_id': u'Ammodramus leconteii',
                           'notes': ''},
 u'Ammodramus maritimus': {'GAP-ITIS_TSN': 179346,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u'Sea

 u'Aneides aeneus': {'GAP-ITIS_TSN': 173699,
                     'GAP-ITIS_match_notes': 'ITIS valid match.',
                     'GAP_common_name': u'Green Salamander',
                     'GAP_scientific_name': 'Aneides aeneus',
                     'GAP_species_code': 'aGRSAx',
                     'GAP_taxa_notes': nan,
                     'SGCN-ITIS_TSN': 173699,
                     'SGCN_commmon_name': u'Green Salamander',
                     'SGCN_id': u'Aneides aeneus',
                     'notes': ''},
 u'Aneides ferreus': {'GAP-ITIS_TSN': 173700,
                      'GAP-ITIS_match_notes': 'ITIS valid match.',
                      'GAP_common_name': u'Clouded Salamander',
                      'GAP_scientific_name': 'Aneides ferreus',
                      'GAP_species_code': 'aCLSAx',
                      'GAP_taxa_notes': nan,
                      'SGCN-ITIS_TSN': 173700,
                      'SGCN_commmon_name': u'Clouded Salamander',
                      'SG

                            'notes': ''},
 u'Archilochus colubris': {'GAP-ITIS_TSN': 178032,
                           'GAP-ITIS_match_notes': 'ITIS valid match.',
                           'GAP_common_name': u'Ruby-throated Hummingbird',
                           'GAP_scientific_name': 'Archilochus colubris',
                           'GAP_species_code': 'bRTHUx',
                           'GAP_taxa_notes': nan,
                           'SGCN-ITIS_TSN': 178032,
                           'SGCN_commmon_name': u'Ruby-throated Hummingbird',
                           'SGCN_id': u'Archilochus colubris',
                           'notes': ''},
 u'Arctocephalus townsendi': {'GAP-ITIS_TSN': 180636,
                              'GAP-ITIS_match_notes': 'ITIS valid match.',
                              'GAP_common_name': u'Guadalupe Fur Seal',
                              'GAP_scientific_name': 'Arctocephalus townsendi',
                              'GAP_species_code': 'mGFSEx',
   

                           'GAP_taxa_notes': nan,
                           'SGCN-ITIS_TSN': 176610,
                           'SGCN_commmon_name': u'Upland Sandpiper',
                           'SGCN_id': u'Bartramia longicauda',
                           'notes': ''},
 u'Bassariscus astutus': {'GAP-ITIS_TSN': 180577,
                          'GAP-ITIS_match_notes': 'ITIS valid match.',
                          'GAP_common_name': u'Ringtail',
                          'GAP_scientific_name': 'Bassariscus astutus',
                          'GAP_species_code': 'mRINGx',
                          'GAP_taxa_notes': nan,
                          'SGCN-ITIS_TSN': 180577,
                          'SGCN_commmon_name': u'Ringtail',
                          'SGCN_id': u'Bassariscus astutus',
                          'notes': ''},
 u'Batrachoseps campi': {'GAP-ITIS_TSN': 173707,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_n

...

                           'GAP_common_name': u'Marsh Rabbit',
                           'GAP_scientific_name': 'Sylvilagus palustris',
                           'GAP_species_code': 'mMARAx',
                           'GAP_taxa_notes': nan,
                           'SGCN-ITIS_TSN': 180120,
                           'SGCN_commmon_name': u'Marsh Rabbit',
                           'SGCN_id': u'Sylvilagus palustris',
                           'notes': ''},
 u'Sylvilagus palustris hefneri': {'GAP-ITIS_TSN': 202349,
                                   'GAP-ITIS_match_notes': 'ITIS valid match.',
                                   'GAP_common_name': u'Lower Keys Rabbit',
                                   'GAP_scientific_name': 'Sylvilagus palustris hefneri',
                                   'GAP_species_code': 'mMARAh',
                                   'GAP_taxa_notes': nan,
                                   'SGCN-ITIS_TSN': 202349,
                                   'SGCN_commmo

 u'Taricha rivularis': {'GAP-ITIS_TSN': 173621,
                        'GAP-ITIS_match_notes': 'ITIS valid match.',
                        'GAP_common_name': u'Red-bellied Newt',
                        'GAP_scientific_name': 'Taricha rivularis',
                        'GAP_species_code': 'aRBNEx',
                        'GAP_taxa_notes': nan,
                        'SGCN-ITIS_TSN': 173621,
                        'SGCN_commmon_name': u'Red-bellied Newt',
                        'SGCN_id': u'Taricha rivularis',
                        'notes': ''},
 u'Taricha torosa': {'GAP-ITIS_TSN': 173622,
                     'GAP-ITIS_match_notes': 'ITIS valid match.',
                     'GAP_common_name': u'California Newt',
                     'GAP_scientific_name': 'Taricha torosa',
                     'GAP_species_code': 'aCFNEx',
                     'GAP_taxa_notes': nan,
                     'SGCN-ITIS_TSN': 173622,
                     'SGCN_commmon_name': u'California Newt',
    

 u'Trichechus manatus': {'GAP-ITIS_TSN': 180684,
                         'GAP-ITIS_match_notes': 'ITIS valid match.',
                         'GAP_common_name': u'West Indian Manatee',
                         'GAP_scientific_name': 'Trichechus manatus',
                         'GAP_species_code': 'mWIMAx',
                         'GAP_taxa_notes': nan,
                         'SGCN-ITIS_TSN': 180684,
                         'SGCN_commmon_name': u'West Indian Manatee',
                         'SGCN_id': u'Trichechus manatus',
                         'notes': ''},
 u'Trimorphodon biscutatus': {'GAP-ITIS_TSN': 174291,
                              'GAP-ITIS_match_notes': 'ITIS valid match.',
                              'GAP_common_name': u'Western Lyresnake',
                              'GAP_scientific_name': 'Trimorphodon biscutatus',
                              'GAP_species_code': 'rWELYx',
                              'GAP_taxa_notes': nan,
                             

                        'GAP_taxa_notes': 'ITIS and NatureServe recognize the splitting of 8 genera (Notocitellus, Otospermophilus, Callospermophilus, Ictidomys, Poliocitellus, Xerospermophilus, and Urocitellus) from Spermophilus (which is now restricted to Eurasia) based on Helgen et al. (2009).\r\nSGW 29 September 2017',
                        'SGCN-ITIS_TSN': 930317,
                        'SGCN_commmon_name': u"Merriam's Ground Squirrel",
                        'SGCN_id': u'Urocitellus canus',
                        'notes': ''},
 u'Urocitellus elegans': {'GAP-ITIS_TSN': 930319,
                          'GAP-ITIS_match_notes': 'ITIS valid match.',
                          'GAP_common_name': u'Wyoming Ground Squirrel',
                          'GAP_scientific_name': 'Urocitellus elegans',
                          'GAP_species_code': 'mWYSQx',
                          'GAP_taxa_notes': 'NatureServe does not recognize the subspecies Urocitellus elegans aureus or U. E. elegans

                   'notes': ''},
 u'Vulpes vulpes': {'GAP-ITIS_TSN': 180604,
                    'GAP-ITIS_match_notes': 'ITIS valid match.',
                    'GAP_common_name': u'Red Fox',
                    'GAP_scientific_name': 'Vulpes vulpes',
                    'GAP_species_code': 'mREFOx',
                    'GAP_taxa_notes': nan,
                    'SGCN-ITIS_TSN': 180604,
                    'SGCN_commmon_name': u'Red Fox',
                    'SGCN_id': u'Vulpes vulpes',
                    'notes': ''},
 u'Vulpes vulpes necator': {'GAP-ITIS_TSN': 726953,
                            'GAP-ITIS_match_notes': 'ITIS valid match.',
                            'GAP_common_name': u'Sierra Nevada Red Fox',
                            'GAP_scientific_name': 'Vulpes vulpes necator',
                            'GAP_species_code': 'mREFOn',
                            'GAP_taxa_notes': nan,
                            'SGCN-ITIS_TSN': 726953,
                            'SGCN_com